In [100]:
import numpy as np
import pandas as pd
import requests
import json
import keyring
import time
import datetime
from requests.structures import CaseInsensitiveDict

* [Docs for the API](https://certificationapi.oshwa.org/documentation#section/Introduction)
* [The website version is here](https://certification.oshwa.org/list.html)
* Step one, get a token!
    * Fill out the form [here](https://certificationapi.oshwa.org/)
    * There is a pop-up that will have a big long string of letters and nubmers

* Step two, let's add that token to our our keyring
    * On Linux install keyring with, `sudo apt install keyring`
    * Copy the api key and then add it to keyring by running  `keyring set oshwa_api NoName` and pasting the api key
    * To check the keyring run `keyring set oshwa_api NoName`
    * Add it to your password manager if you use one. 
* Next we'll add the keyring safely to our notebook

In [4]:
# We'll get our first key
oshwa_key = keyring.get_password("oshwa_api","NoName")
if oshwa_key is None:
    print('Failed to get API key')
else:
    print("Got API Key")

Got API Key


# Now we're going to make our first request. We'll need our endpoint and a token.
# [This tool is really helpful for figuring out request formats](https://reqbin.com/req/python/5k564bhv/get-request-with-bearer-token-authorization-header) 

In [85]:
url = "https://certificationapi.oshwa.org/api/projects/"
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer {0}".format(oshwa_key)
params = {"limit":1500,"page":3}
resp = requests.get(url, headers=headers, params=params)
print(resp.status_code)
if resp.status_code == 200:
    cert_list = resp.json()
    print(len(cert_list))

200
1000


In [106]:
# Now we're going to convert this data to a pandas dataframe and clean up the time
df = pd.DataFrame(data=cert_list)
df.to_csv("RawData.csv")
df['certificationDate'] = pd.to_datetime(df['certificationDate'], format="%Y-%m-%d %H:%M:%S")
df.head()

,oshwaUid,responsibleParty,country,publicContact,projectName,projectWebsite,projectVersion,projectDescription,primaryType,additionalType,projectKeywords,citations,documentationUrl,hardwareLicense,softwareLicense,documentationLicense,certificationDate,previousVersions
0,MX000027,Nehenemi Labs,Mexico,nehenemilabs@gmail.com,ULTFreezer monitor,https://github.com/nehenemilabs/ULTFreezer-mon...,0.2b,A quick and cheap wifi enabled kit for monitor...,Science,"[Electronics, IOT, Science, Tool]",[vaccines ULTs freezer monitor wifi],[{'url': 'https://www.nodemcu.com/index_en.htm...,https://github.com/nehenemilabs/ULTFreezer-mon...,CERN,GPL,CC BY-SA,2021-03-18 00:00:00-04:00,NaN
1,US001062,System76,United States of America,info@system76.com,System76 Launch Configurable Keyboard,https://system76.com/accessories/launch,1.0,The System76 Launch Configurable Keyboard is d...,Electronics,NaN,"[Keyboard, QMK, Linux]",[],https://github.com/system76/launch,Other,GPL,CC BY-SA,2021-02-17 00:00:00-05:00,NaN
2,ID000006,SENTSOR Electronics,Indonesia,sentsor.tech@gmail.com,SENTSOR Uno Plus,https://github.com/adamalfath/sentsor-uno-plus,1.0,Your beloved classic Arduino Uno development b...,Electronics,"[Education, Electronics, IOT]","[SENTSOR, DEVELOPMENT BOARD]",[],https://github.com/adamalfath/sentsor-uno-plus,Other,No software,CC BY-SA,2021-03-18 00:00:00-04:00,NaN
3,UK000026,Nebra Ltd,United Kingdom,sales@nebra.com,LoRa Gateway mPCIe Module (SX1302/3),https://github.com/NebraLtd/LoRa-mPCIe,1.0,Open source LoRa gateway concentrator in mPCIe...,IOT,"[Electronics, Home Connection, IOT, Manufactur...","[LoRa, LoRaWAN, IoT, mPCIe, lora concentra...",[{'url': 'https://www.semtech.com/products/wir...,https://github.com/NebraLtd/LoRa-mPCIe,Other,Apache,CC BY,2021-03-18 00:00:00-04:00,NaN
4,CL000002,OTTAA Project (Legal name: Comunicación Aume...,Chile,info@ottaaproject.com,OTTAA Sip&Puff,https://ottaa-project.github.io/docs/sipAndPuff,1.0,OTTAA Sip&Puff switch control is part of the O...,Other,"[3D Printing, Education, Wearables]","[Accessibility, HID, Adapted mouse, Sip, Pu...",[],https://ottaa-project.github.io/docs/sipAndPuff,CERN,No software,CC BY-SA,2021-03-18 00:00:00-04:00,NaN


In [116]:
# Let's now just pick certifications from 2020
start = datetime.datetime(2020,1,1,tzinfo=datetime.timezone.utc)
stop = datetime.datetime(2020,12,31,tzinfo=datetime.timezone.utc)
df2020 = df.loc[(df['certificationDate'] >= start) & 
                (df['certificationDate'] <  stop)] 
print("{0} projects from 2020.".format(len(df2020)))

770 projects from 2020.


In [120]:
print("Unique countries {0}".format(df2020.country.unique()))
print("Unique people {0}".format(df2020.responsibleParty.unique()))

Unique countries ['United States of America' 'Croatia' 'Mauritius' 'Switzerland'
 'Indonesia' 'United Kingdom' 'Brazil' 'Germany' 'Sweden' 'India'
 'Guatemala' 'Australia' 'Spain' 'China' 'Ireland' 'Czech Republic'
 'Bulgaria' 'Russia' 'Canada' 'Thailand' 'Mexico' 'Netherlands' 'France'
 'Lithuania' 'Costa Rica' 'El Salvador' 'Finland' 'Taiwan' 'Greece'
 'Portugal' 'Sri Lanka' 'Poland']
Unique people ['System76' 'TAVU d.o.o.' 'SparkFun Electronics' 'TECKNOLOGG'
 'Whitebox Labs' 'SENTSOR ELECTRONICS' 'Nebra Ltd' 'Franzininho'
 'Neil Hardy' 'Centro de Tecnologia Acadêmia (IF/UFRGS)' 'Keith Burzinski'
 'Thore Krug' 'Anuradha Reddy' 'Adafruit Industries, LLC'
 'Upside Down Labs' 'Angel Isidro' 'Freetronics' 'Blackhand Dynamics, SL'
 '深圳市嘉禾三维科技有限公司' 'Brian Lough' 'Watterott electronic'
 'Prusa Research a.s.' 'bmc::labs GmbH' 'HiMinds' 'Dimitar Dimitrov'
 'Luxonis Holding Corporation' 'Andrey Lamchenko' 'BIEM ETC design'
 'Thorsten Jaeger' 'OLIMEX Ltd' 'Jacob Proctor' 'Maker and IoT Ideas'
 